In [5]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, Bounds, OptimizeResult
from CIRModel import BasicCIRModel
import sys; sys.path.insert(0, '..')
from scipy.optimize import Bounds
from scipy.stats import norm

In [10]:
from scipy.optimize import differential_evolution

class Calibration:
    def __init__(self, data, dt, model_class, initial_params):
        self.data = data
        self.dt = dt
        self.model_class = model_class
        self.initial_params = initial_params

    def _nlog_likelihood(self, params, *args):
        updated_params = self.initial_params.copy()
        updated_params.update(dict(zip(args, params)))
        model = self.model_class(**updated_params)
        loglikelihood = 0
        for i in range(1, len(self.data)):
            current_rate = self.data[i-1]
            next_rate = self.data[i]
            predicted_rate = model.next_rate(current_rate, self.dt)
            variance = max(model.sigma**2 * max(current_rate, 0) * self.dt, 1e-8)
            probability_density = norm.pdf(next_rate, loc=predicted_rate, scale=np.sqrt(variance))
            loglikelihood -= np.log(probability_density + 1e-10)
        return loglikelihood

    def calibrate(self, bounds, params):
        result = differential_evolution(
            lambda x: self._nlog_likelihood(x, *params),
            bounds=bounds,
            strategy='best1bin',
            maxiter=1000,
            popsize=15,
            tol=0.01,
            mutation=(0.5, 1),
            recombination=0.7,
            disp=True
        )
        return result

In [11]:
# calibrate using 3mon treasury bill YTM for the past 6 months
df_1_yc = pd.read_csv("DGS3MO.csv")
# data cleaning
df_1_yc.index = pd.to_datetime(df_1_yc.DATE)
df_1_yc = df_1_yc.dropna().drop(columns=["DATE"])
df_1_yc.DGS3MO = df_1_yc.DGS3MO.replace(".", np.nan).astype(float)/100
df_1_yc = df_1_yc.dropna()
# filter out calibration period
df_1_yc = df_1_yc['2023-12-15':'2024-03-15']
data = df_1_yc['DGS3MO'].values

# implementation
calibrator = Calibration(
    data=data,
    dt=1/252,
    model_class=BasicCIRModel,
    initial_params={'kappa': 0.5, 'mu_r': 0.03, 'sigma': 0.03}
)

bounds = Bounds([0.01, 0.01, 0.01], [1, 1, 1])  # Specify parameter bounds
params = ['kappa', 'mu_r', 'sigma']  # Parameters to optimize
result = calibrator.calibrate(bounds=bounds, params=params)

print("Optimized Parameters:")
for param, value in zip(params, result.x):
    print(f"{param}: {value:.4f}")

print(f"\nObjective Function Value: {result.fun:.6f}")
print(f"Success: {result.success}")
print(f"Message: {result.message}")

differential_evolution step 1: f(x)= -412.1691815230636
differential_evolution step 2: f(x)= -453.8782323156116
differential_evolution step 3: f(x)= -472.18900910986986
differential_evolution step 4: f(x)= -472.18900910986986
differential_evolution step 5: f(x)= -472.86941230820736
differential_evolution step 6: f(x)= -472.86941230820736
differential_evolution step 7: f(x)= -472.86941230820736
differential_evolution step 8: f(x)= -473.9895019841692
differential_evolution step 9: f(x)= -475.1269737995221
differential_evolution step 10: f(x)= -475.1269737995221
differential_evolution step 11: f(x)= -475.1269737995221
differential_evolution step 12: f(x)= -475.1269737995221
differential_evolution step 13: f(x)= -475.1269737995221
Polishing solution with 'L-BFGS-B'
Optimized Parameters:
kappa: 0.0565
mu_r: 0.0996
sigma: 0.0124

Objective Function Value: -475.126974
Success: True
Message: Optimization terminated successfully.
